In [1]:
from selenium import webdriver
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options

In [2]:
def search(driver, date):
    """
    Date in mm/dd/yyyy
    """
    driver.get("https://merolagani.com/Floorsheet.aspx")
    date_input = driver.find_element_by_xpath('/html/body/form/div[4]/div[4]/div/div/div[1]/div[4]/input')
    search_btn = driver.find_element_by_xpath('/html/body/form/div[4]/div[4]/div/div/div[2]/a[1]')
    date_input.send_keys(date)
    search_btn.click()

In [3]:
def get_page_table(driver, table_class):
    soup = BeautifulSoup(driver.page_source,'html')
    table = soup.find("table", {"class":table_class})
    tab_data = [[cell.text.replace('\r', '').replace('\n', '') for cell in row.find_all(["th","td"])]
                        for row in table.find_all("tr")]
    df = pd.DataFrame(tab_data)
    return df

In [4]:
def scrape_data(driver, date):
    start_time = datetime.now()
    search(driver, date = date)
    df = pd.DataFrame()
    while True:
        page_table_df = get_page_table(driver, table_class="table table-bordered table-striped table-hover sortable")
        df = df.append(page_table_df, ignore_index = True)
        try:
            next_btn = driver.find_element_by_link_text('Next')
            driver.execute_script("arguments[0].click();", next_btn)
        except NoSuchElementException:
            break
    print(f"Time taken to scrape: {datetime.now() - start_time}")    
    return df

In [5]:
def clean_df(df):
    new_df = df.drop_duplicates(keep='first') # Dropping Duplicates
    new_header = new_df.iloc[0] # grabing the first row for the header
    new_df = new_df[1:] # taking the data lower than the header row
    new_df.columns = new_header # setting the header row as the df header
    new_df.drop(["#"], axis=1, inplace=True)
    new_df["Rate"] = new_df["Rate"].apply(lambda x:float(x.replace(",", ""))) # Convert Rate to Float
    new_df["Amount"] = new_df["Amount"].apply(lambda x:float(x.replace(",", ""))) # Convert Amount to Float
    return new_df

In [6]:
options = Options()
options.headless = True
driver = webdriver.Chrome(chrome_options=options) # Start Browser

date = datetime.today().strftime('%m/%d/%Y') # Get today's date
search(driver, date) # Search the webpage
df = scrape_data(driver, date) # Scraping
final_df = clean_df(df) # Cleaning

/home/suyog/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


Time taken to scrape: 0:07:19.085173


NameError: name 'new_df' is not defined

In [7]:
final_df.head()

,Transact. No.,Symbol,Buyer,Seller,Quantity,Rate,Amount
1,2020121604001499,BBC,45,34,10,2793.0,27930.0
2,2020121604001489,BBC,57,33,10,2793.0,27930.0
3,2020121604001468,BBC,49,34,10,2800.0,28000.0
4,2020121604001437,BBC,14,34,20,2800.0,56000.0
5,2020121604001436,BBC,42,34,15,2800.0,42000.0


In [9]:
file_name = date.replace("/", "_")
final_df.to_csv(f"data/{file_name}.csv", index=False) # Save file